The objective is to develop a model that predicts the composer of a piece of classical music, represented in CSV format that was converted directly from MIDI data.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

np.random.seed(0)

All the categories in the original csv is as follows: tick, type, time, meta, track, numerator, denominator, clocks_per_click, notated_32nd_notes_per_beat, key, tempo, control, value, channel, program, note, velocity

--------------

PREPROCESSING 1:
 * NOTE: time column represents the time to wait since the last note played on that channel. This information is fully duplicated in the form of a linear timeline represented by the tick column
 * NOTE: note_off and note_on + velocity of 0 are equivalent

The only useful ones for us are: tick (temporal dimension), type (note_on and note_off are the only one that matters), channel, note (pitch), velocity (loudness)

--------------

PROPROCESSING 2:
For training, the notes will be represented as follows:
 * a note-based DS will be generated. Each row is a note containing its start-tick, duration, pitch and velocity. Shape: (NOTES, 4)
 * a chord-based DS will then be generated. All notes that have the same start-tick, duration and velocity will be grouped together, up to 4. Shape: (CHORDS, 4, 4). Zeros will be used as padding for chords less than 4 notes.

Each song piece may have a maximum CHORDS length of 300. This is just so we have consistent training and testing data shape, in reality it should theoritically work for other sizes as well.

Both will be attempted for training, I suspect the chord based method might have advantages but I'm not sure.

--------------

DATA AUGMENTATION: we can segment the data in many more ways, eg chords 0-300, 10-310, ... , 5900-6200 etc

--------------

DATA EXPLORATION:
 * statistics on chord formation, number of notes sharing the same tick, overall velocity, duration and pitch running average/std deviation
 * outliers in terms of song length, too many notes per chord, long pauses between notes

In [7]:
from dictionary_array import Dict_Array

def process_notes(notes):
    FEATURES_PER_CHORD = 129
    pending_notes = {}
    processed_notes = Dict_Array(list_type='list')

    #print(f'notes: {notes}')

    for note in notes:
        if note['type'] not in ('note_on', 'note_off'):
            continue
        note_id = f"{note['note']}+{note['channel']}"
        if note_id in pending_notes and (note['velocity'] == 0 or note['type'] == 'note_off'):
            pn = pending_notes[note_id]
            processed_note = {'tick': pn['tick'], 'duration': note['tick'] - pn['tick'], 'note': pn['note'], 'channel': pn['channel'], 'velocity': pn['velocity']}
            processed_notes.add(f"{note['tick']}+{note['channel']}+{processed_note['duration']}", processed_note)
        elif note['type'] == 'note_on' and note['velocity'] > 0:
            pending_notes.update({note_id:note})

    
    # at this point, all chords are organized together within processed_notes, they just require sorting before insertion into chords
    # TODO: convert processed_notes into a key held one hot encoding format.
    
    sorted_keys = list(processed_notes.dictionary.keys())
    sorted_keys.sort()
    chords = np.zeros((len(sorted_keys), FEATURES_PER_CHORD))

    for i, key in enumerate(sorted_keys):
        notes = processed_notes.get(key)
        for j, note in enumerate(notes):
            chords[i, note['note'] + 1] = note['velocity'] + 100

    #print(f'chords: {chords}')

    return chords
    

# augment into multiple chord sets
def split_chords_into_sets(chords):
    FEATURES_PER_CHORD = len(chords[0])
    SKIP_SIZE = 100
    CHORD_SET_SIZE = 25
    start_index = 0
    num_chord_sets = int(max(1, np.ceil((len(chords) - CHORD_SET_SIZE) / SKIP_SIZE)) + 1)
    chord_sets = []
    for i in range(num_chord_sets):
        end_index = min(start_index + CHORD_SET_SIZE, len(chords))
        if end_index <= start_index:
            break

        chord_set = np.zeros((CHORD_SET_SIZE, FEATURES_PER_CHORD))
        chord_set[0:end_index - start_index] = chords[start_index:end_index]
        chord_sets.append(chord_set)

        start_index += SKIP_SIZE

    #print(f'chord_sets: {chord_sets}')

    return chord_sets


def to_int(x):
    if x is None or x == '':
        x = -1
    return int(x)


def import_data(data_dir, folders):
    # folders respresent the directories to import, leave as none to import all
    CATEGORIES = len(folders)

    total_items = 0
    categories = 0
    for subdir in os.listdir(data_dir):
        if folders is not None and subdir not in folders:
            continue
        subdir_path = os.path.join(data_dir, subdir)
        if not os.path.isdir(subdir_path):
            continue
        categories += 1
        total_items += len(os.listdir(subdir_path))

    X_temp = []
    Y_temp = []

    chords = []
    
    category_counter = 0
    item_counter = 0
    # List all subfolders
    for subdir in os.listdir(data_dir):
        # if folders list parameter is specified, only continue if this directory is included in it
        if folders is not None and subdir not in folders:
            continue
        subdir_path = os.path.join(data_dir, subdir)
        # ensure this subdirectory is a folder
        if not os.path.isdir(subdir_path):
            continue
        print(f'reading directory {subdir}')

        # process each file in the subdirectory
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)

            all_notes = []

            with open(file_path, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                header = next(csvreader)
                IDX_TICK = header.index('tick')
                IDX_TYPE = header.index('type')
                IDX_CHAN = header.index('channel')
                IDX_NOTE = header.index('note')
                IDX_VELO = header.index('velocity')

                for row in csvreader:
                    all_notes.append({'tick': to_int(row[IDX_TICK]), 
                                      'type': row[IDX_TYPE], 
                                      'channel': to_int(row[IDX_CHAN]), 
                                      'note': to_int(row[IDX_NOTE]), 
                                      'velocity': to_int(row[IDX_VELO])})

            chords = process_notes(all_notes)
            chord_sets = split_chords_into_sets(chords)
            X_temp.extend(chord_sets)
            
            category_one_hot = [0] * CATEGORIES
            category_one_hot[category_counter] = 1
            categories_extension = []
            for _ in range(len(chord_sets)):
                categories_extension.append(category_one_hot)
            Y_temp.extend(categories_extension)

            item_counter += 1

        category_counter += 1

    X = np.array(X_temp)
    Y = np.array(Y_temp)
    print(f'X shape: {X.shape} Y shape: {Y.shape}')
    print(f'CATEGORY STATS:')
    for i in range(Y.shape[1]):
        print(f'  {i}: {np.sum(Y[:,i])}')
    return X, Y

VISUALIZATION:

take a chordset and print it in music sheet style to see if it looks reasonable. Compare to original. This helps with outlier and anomaly detection, and catched mistakes that might have been present in the original dataset. Outliers can then be removed when necessary, such as a note that is (mistakenly) held for the entire piece.

In [8]:
from sklearn.model_selection import train_test_split

data_path = '../../Data/music/midi/'

FEATURES_PER_CHORD = 129
CATEGORIES = ['Chopin', 'Vivaldi']
#CATEGORIES = ["Chopin", "Vivaldi", "Scarlatti", "Haendel"]
n_categories = len(CATEGORIES)

X, Y = import_data(data_path, CATEGORIES)
X = X.astype(np.float32)
Y = Y.astype(np.float32)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
print(f'xtrain: {x_train.shape} ytrain: {y_train.shape} xtest: {x_test.shape} ytest: {y_test.shape}')

reading directory Chopin
reading directory Vivaldi
X shape: (5944, 25, 129) Y shape: (5944, 2)
CATEGORY STATS:
  0: 2571
  1: 3373
xtrain: (5349, 25, 129) ytrain: (5349, 2) xtest: (595, 25, 129) ytest: (595, 2)


In [9]:
#TODO: make RNN model and visualization. I have preprocessed the data into an organized, uniform format ready to be used. May need to build it custom from linear

def get_category(output):
    return torch.argmax(torch.abs(output))
    #return torch.argmax(output)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.in2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        #print(f'input: {input.size()}, hidden: {hidden.size()}')
        combined = torch.cat((input, hidden), 1)
        hidden = self.in2h(combined)
        output = self.h2out(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        #return torch.zeros(1, self.hidden_size)
        return (torch.rand(1, self.hidden_size) - 0.5) * 0.02
    
x_train = torch.from_numpy(x_train)
x_train = torch.unsqueeze(x_train, 2)
y_train = torch.from_numpy(y_train)
y_train = torch.unsqueeze(y_train, 1)
y_train = y_train.type(torch.LongTensor)

x_test = torch.from_numpy(x_test)
x_test = torch.unsqueeze(x_test, 2)
y_test = torch.from_numpy(y_test)
y_test = torch.unsqueeze(y_test, 1)
y_test = y_test.type(torch.LongTensor)

In [10]:
learning_rate = 0.001
criterion = nn.NLLLoss()
n_hidden = 128
rnn = RNN(FEATURES_PER_CHORD, n_hidden, n_categories)

def train(chord_set, category):
    hidden = rnn.initHidden()
    rnn.zero_grad()

    for i in range(chord_set.size()[0]):
        output, hidden = rnn(chord_set[i], hidden)

    #print(f'output: {output} category: {category}')
    loss = criterion(output[0], category[0])
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        clip_value = 0.1
        p.register_hook(lambda grad: torch.clamp(grad, -clip_value, clip_value))
        #print(f'p data: {p.data} grad: {p.grad.data}')
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()


def test(testX, testY):
    correct = 0
    for index, x in enumerate(testX):
        hidden = rnn.initHidden()
        for i in range(chord_set.size()[0]):
            output, hidden = rnn(x[i], hidden)
        if get_category(output) == get_category(testY[index]):
            correct += 1
    return correct / len(testX)


n_epoch = 10
all_losses = []
current_loss = 0
correct = 0
total = 0

for epoch in range(n_epoch):
    plot_interval = max(1, int(len(x_train) / 20))
    for i in range(len(x_train)):
        chord_set = x_train[i]
        category = y_train[i]
        output, loss = train(chord_set, category)
        current_loss += loss/plot_interval

        total += 1
        if get_category(output) == torch.argmax(category):
            #print(f'carrot: OUTPUT {torch.argmax(output)}')
            correct += 1
        #else:
           # print(f'INCORRECT LMAO, output: {output} answer: {category}')

        if i % plot_interval == 0 and i != 0:
            print(f'epoch {epoch}, iteration {i}/{len(x_train)}, loss {current_loss}, {correct}/{total} correct ( {correct/total} )')
            all_losses.append(current_loss)
            current_loss = 0
            correct = 0
            total = 0
            print(f'testing accuracy: {test(x_test, y_test)}')

    correct = 0

epoch 0, iteration 267/5349, loss 2.684558664530926, 137/268 correct ( 0.5111940298507462 )
testing accuracy: 0.5394957983193277
epoch 0, iteration 534/5349, loss 2.1844922330347876, 158/267 correct ( 0.5917602996254682 )
testing accuracy: 0.5596638655462185
epoch 0, iteration 801/5349, loss 1.9355258952087893, 166/267 correct ( 0.6217228464419475 )
testing accuracy: 0.5915966386554622
epoch 0, iteration 1068/5349, loss 2.062102575800115, 145/267 correct ( 0.5430711610486891 )
testing accuracy: 0.6067226890756302
epoch 0, iteration 1335/5349, loss 1.4682692638281007, 185/267 correct ( 0.6928838951310862 )
testing accuracy: 0.6235294117647059
epoch 0, iteration 1602/5349, loss 1.7341596187513861, 153/267 correct ( 0.5730337078651685 )
testing accuracy: 0.6470588235294118
epoch 0, iteration 1869/5349, loss 1.6215140293884756, 165/267 correct ( 0.6179775280898876 )
testing accuracy: 0.6470588235294118
epoch 0, iteration 2136/5349, loss 1.8221420661626293, 156/267 correct ( 0.5842696629213

KeyboardInterrupt: 